In [21]:
import pandas as pd
import numpy as np
import pathlib 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
import warnings
import pandas as pd
import argparse

from dataset_paths import *

warnings.filterwarnings('ignore')

output_version = "20241229"
outdir = "/media/hieunguyen/HNSD_mini/outdir"

mode = "all"
PROJECT = "combine_ctcandi_ichorcna"

for dataset_name in ["VALIDATION", "LOD", "SPIKE_IN"]:
    metadata = pd.read_csv(f"./metadata/{dataset_name}.csv")

    for input_cancer_class in ["CRC", "Liver", "Lung", "Breast", "pan_cancer"]:
        path_to_main_output = os.path.join(outdir, PROJECT, output_version, dataset_name)
        path_to_07_output = os.path.join(path_to_main_output, f"07_output_{mode}", input_cancer_class)
        path_to_08_output = os.path.join(path_to_main_output, f"08_output_{mode}", input_cancer_class)
        os.system(f"mkdir -p {path_to_08_output}")
        all_read_classification_files = [item for item in pathlib.Path(path_to_07_output).glob("*.raw.read_classification.csv")]
        all_candi_read_files = [item for item in pathlib.Path(path_to_07_output).glob("*candi_reads.csv")]

        if dataset_name == "LOD":
            convert_ratio = {
            '50' : 0.5, 
            '100': 1, 
            '0.5': 0.005, 
            '25': 0.25, 
            '15': 0.15, 
            '5': 0.05, 
            '1': 0.01, 
            'HC': 0
            }
            metadata = metadata[metadata["Sample"].duplicated() == False]
            metadata["SampleID"] = metadata["Sample"].values
            metadata.columns = ["ichorCNA" if item == "Actual tumor_fraction_ichorCNA" else item for item in metadata.columns]
            metadata["spike_in_ratio"] = metadata["spike-in"].apply(lambda x: convert_ratio[x])
            metadata['Label'] = metadata["LABEL"].apply(lambda x: "CRC" if x == "Colorectal cancer" else x.split(" ")[0])
            metadata["spike_in_label"] = metadata["Simulated TF"].apply(lambda x: "Control" if x == "Healthy-control" else input_cancer_class)
            metadata["Label"] = metadata[["Label", "spike_in_label"]].apply(lambda x: x[0] if x[1] != "Control" else "Control", axis = 1)
            metadata = metadata[["SampleID", "ichorCNA", "spike_in_ratio", "Label"]]
        elif dataset_name == "VALIDATION":
            metadata = metadata[["SampleID", "ichorCNA"]]
            all_candi_read_files = [item for item in all_candi_read_files if item.name.replace(".candi_reads.csv", "") in metadata["SampleID"].unique()]
            all_read_classification_files = [item for item in all_read_classification_files if item.name.replace(".raw.read_classification.csv", "") in metadata["SampleID"].unique()]
        elif dataset_name == "SPIKE_IN":
            metadata = metadata[["SampleID", "Spike_in_label", "Spike_in_ratio", "ichorCNA"]]
            metadata["Spike_in_ratio"] = metadata["Spike_in_ratio"].apply(lambda x: x/100)
            metadata.columns = ["SampleID", "Label", "spike_in_ratio", "ichorCNA"]
            metadata = metadata[["SampleID", "ichorCNA", "spike_in_ratio", "Label"]]
        elif dataset_name == "REPORT4":
            metadata = metadata[["SampleID", "ichorCNA", "Label"]]
            metadata = metadata[metadata["ichorCNA"].isna() == False]

        print(f"Number of candi read files: {len(all_candi_read_files)}")
        print(f"Number of classification files: {len(all_read_classification_files)}")

        if os.path.isfile(os.path.join(path_to_08_output, "feature.csv")) == False:
            fulldf = pd.DataFrame()
            for i in tqdm(range(len(all_candi_read_files))):
                tmp_readdf = pd.read_csv(all_read_classification_files[i], index_col = [0])
                tmp_candidf = pd.read_csv(all_candi_read_files[i], index_col = [0])
                sampleid = all_candi_read_files[i].name.replace(".candi_reads.csv", "")
                raw_count = tmp_readdf.shape[0]
                in_read_count = tmp_readdf[tmp_readdf["read_overlap_rate"] == "in"].shape[0]
                mean_candi_reads = tmp_candidf.candi_reads.mean()
                ratio_raw = mean_candi_reads/raw_count
                ratio_in_read = mean_candi_reads/in_read_count
                tmpdf = pd.DataFrame({"SampleID": sampleid, 
                                    "raw_count": raw_count, 
                                    "in_read_count": in_read_count, 
                                    "mean_candi_reads": mean_candi_reads,
                                    "ratio_raw": ratio_raw,
                                    "ratio_in_read": ratio_in_read}, index = [0])

                fulldf = pd.concat([fulldf, tmpdf], axis = 0)
            fulldf = fulldf.merge(metadata, right_on = "SampleID", left_on = "SampleID")
            fulldf.to_csv(os.path.join(path_to_08_output, "feature.csv"))
        else:
            fulldf = pd.read_csv(os.path.join(path_to_08_output, "feature.csv"), index_col = [0])


Number of candi read files: 427
Number of classification files: 427


100%|██████████| 427/427 [00:50<00:00,  8.38it/s]


Number of candi read files: 427
Number of classification files: 427


 29%|██▉       | 125/427 [00:16<00:39,  7.63it/s]


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

array(['Breast cancer', 'Colorectal cancer', 'Lung cancer',
       'Cervical cancer', 'Gastric cancer', 'Ovarian cancer',
       'Pancreatic cancer', 'Control', 'Endometric cancer',
       'benign breast', 'no lesions', 'multiple benign', 'benign gastric',
       'pre-K', 'other benign lesions', 'benign lung', 'Liver cancer',
       'lymphoma cancer', 'Prostate cancer'], dtype=object)